# 移动

In [1]:
import pandas as pd
import numpy as np
import time
import geopandas as gpd
import mytools

In [ ]:
sit_cmcc=pd.read_csv(open('G:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/输出数据/汇总/移动汇总.csv',encoding='gbk'))

In [ ]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191212.csv',encoding='gbk'))

In [ ]:
sit_cmcc = sit_cmcc.merge(gongcan[[ '小区中文名', '所属地区']].drop_duplicates('小区中文名'),how='left',on='小区中文名')

In [ ]:
sit_cmcc.columns

In [ ]:
res_table = gongcan.groupby(by='所属地区').agg({'小区中文名':'count'})
res_table.rename(columns={'小区中文名':'小区总计'},inplace=True)

res_table = res_table.merge(sit_cmcc.groupby(by='所属地区').agg({'小区中文名':'count'}),left_index=True,right_index=True)
res_table.rename(columns={'小区中文名':'站址总计'},inplace=True)

In [ ]:
list_pin = ['频段_900M','频段_1800M','频段_2100M']

In [ ]:
for pin in list_pin:
    sit_cmcc_use = sit_cmcc[sit_cmcc[pin]==pin.split('_')[1]]
    res_table = res_table.merge(sit_cmcc_use.groupby(by='所属地区').agg({'小区中文名':'count'}),left_index=True,right_index=True)
    res_table.rename(columns={'小区中文名':pin},inplace=True)

In [ ]:
res_table

In [ ]:
def table_sum(df):
    sum_column = df.sum(axis=1)
    sum_column.name='总计'
    sum_row = df.sum()
    sum_row.name='总计'
    df = pd.concat([df,sum_column],axis=1)
    df = df.append(sum_row)
    return df

In [ ]:
table_sum(res_table)

# 联通电信

In [2]:
dianxin = pd.read_excel('G:/1-规划/评估/竞对站点评估/20200617/1-基础数据/3-电信数据/电信汇总.xlsx',encoding = 'gbk')
liantong = pd.read_csv('G:/1-规划/评估/竞对站点评估/20200617/1-基础数据/2-联通数据/联通-汇总-使用.csv',encoding = 'gbk')

In [3]:
dianxin.columns

Index(['基站名称', '站型（宏站/室分/微站）', '频段', '频点', '  更正频点', '经度', '纬度', '区域(区县）',
       '是否独立站址', '厂家', '900-1800-2400'],
      dtype='object')

In [4]:
liantong.columns

Index(['eNodeB ID', '基站名称', '站型（宏站/室分/微站）', '频段', '频点', '经度', '纬度', '区域(区县）',
       '是否独立站址', '厂家', '900-1800-2400'],
      dtype='object')

In [5]:
dianxin_h = dianxin.loc[dianxin['站型（宏站/室分/微站）']=='宏站']
liantong_h = liantong.loc[liantong['站型（宏站/室分/微站）']=='宏站']

In [6]:
liantong_h.shape

(33011, 11)

In [7]:
hubei = gpd.read_file('g:/1-规划/图层/湖北省地市-边界-边界/全省地市边界.TAB',encoding = 'gbk')
hubei = hubei.to_crs({'init':'epsg:4326'})

In [8]:
hubei = mytools.gisn.maps('地市区县')

In [9]:
dianxin_h = dianxin_h.astype(str)

dianxin_h.info()

dianxin_h['经度'] = dianxin_h['经度'].str.strip()
dianxin_h['纬度'] = dianxin_h['纬度'].str.strip()
dianxin_h['纬度'] = dianxin_h['纬度'].str.replace('?','')
dianxin_h['经度'] = pd.to_numeric(dianxin_h['经度'])
dianxin_h['纬度'] = pd.to_numeric(dianxin_h['纬度'])

# dianxin['经度'] = dianxin['经度'].str.strip()
# dianxin['纬度'] = dianxin['纬度'].str.strip()
# liantong['经度'] = liantong['经度'].str.strip()
# liantong['纬度'] = liantong['纬度'].str.strip()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97222 entries, 0 to 114747
Data columns (total 11 columns):
基站名称             97222 non-null object
站型（宏站/室分/微站）     97222 non-null object
频段               97222 non-null object
频点               97222 non-null object
  更正频点           97222 non-null object
经度               97222 non-null object
纬度               97222 non-null object
区域(区县）           97222 non-null object
是否独立站址           97222 non-null object
厂家               97222 non-null object
900-1800-2400    97222 non-null object
dtypes: object(11)
memory usage: 8.9+ MB


In [10]:
dianxin_use = dianxin_h.loc[(dianxin_h['经度']>60) & (dianxin_h['纬度']<60)]
liantong_use = liantong_h.loc[(liantong_h['经度']>60 & (liantong_h['纬度']<60))]

dianxin_p = mytools.gisn.add_points(dianxin_use ,lon='经度',lat='纬度')
liantong_p = mytools.gisn.add_points(liantong_use , lon='经度',lat='纬度')

In [11]:
dianxin_res = gpd.sjoin(hubei,dianxin_p)
liantong_res = gpd.sjoin(hubei,liantong_p)

dianxin_res.drop(['geometry','index_right'],axis=1,inplace = True)
liantong_res.drop(['geometry','index_right'],axis=1,inplace = True)

In [12]:
liantong_res.drop_duplicates('基站名称').shape

(32463, 13)

In [14]:
dianxin_res['地市'] = dianxin_res['地市'].str.replace('市','')
liantong_res['地市'] = liantong_res['地市'].str.replace('市','')

dianxin_res_groupby = dianxin_res.groupby(by = '900-1800-2400')
liantong_res_groupby = liantong_res.groupby(by = '900-1800-2400')

In [16]:
dishi = dianxin_res.groupby(by = '地市').agg({'基站名称':'count'})
dishi.columns = ['电信小区']
liantong_linshi_1=liantong_res.groupby(by = '地市').agg({'基站名称':'count'})
liantong_linshi_1.columns = ['联通小区']
dishi['联通小区']=0
dishi['联通小区'].update(liantong_linshi_1['联通小区'])

In [18]:
dianxin_res_qc1 = dianxin_res.drop_duplicates('基站名称')
dianxin_linshi_2 = dianxin_res_qc1.groupby(by = ['地市']).agg({'地市':'count'})
dianxin_linshi_2.columns = ['电信总站址']
dishi['电信总站址'] = 0
dishi['电信总站址'].update(dianxin_linshi_2['电信总站址'])

In [19]:
liantong__res_qc1 = liantong_res.drop_duplicates('基站名称')
liantong_linshi_2 = liantong__res_qc1.groupby(by = ['地市']).agg({'地市':'count'})
liantong_linshi_2.columns = ['联通总站址']
dishi['联通总站址'] = 0
dishi['联通总站址'].update(liantong_linshi_2['联通总站址'])

In [20]:
dishi

,电信小区,联通小区,电信总站址,联通总站址
地市,,,,
十堰,4377,2369,3001,2334
咸宁,6712,1408,2747,1405
天门,503,689,483,684
孝感,7686,2284,2168,2284
宜昌,3572,2575,3368,2548
恩施,3849,4200,3485,4197
武汉,24441,6744,6835,6331
江汉,1594,778,468,764
潜江,487,652,465,647


In [21]:
for name,dianxin_t in dianxin_res_groupby:
    dianxin_res_qc = dianxin_t.drop_duplicates('基站名称')
    dianxin_linshi_2 = dianxin_res_qc.groupby(by = ['地市']).agg({'地市':'count'})
    dianxin_linshi_2.columns = [name+'电信总站址']
    dishi[name+'电信总站址'] = 0
    dishi[name+'电信总站址'].update(dianxin_linshi_2[name+'电信总站址'])

for name,liantong_t in liantong_res_groupby:
    liantong_res_qc = liantong_t.drop_duplicates('基站名称')
    liantong_linshi_2 = liantong_res_qc.groupby(by = ['地市']).agg({'地市':'count'})
    liantong_linshi_2.columns = [name+'联通总站址']
    dishi[name+'联通总站址'] = 0
    dishi[name+'联通总站址'].update(liantong_linshi_2[name+'联通总站址'])

In [ ]:
dishi

In [22]:
a= dishi.sum()
a.name = '总计'
dishi =dishi.append(a)

In [ ]:
dishi

In [23]:
dishi.to_clipboard()

In [ ]:
dishi.iloc[0:1,0:2]